In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the csv
df = pd.read_csv("compensation_gender")
df.head()

,totalyearlycompensation,basesalary,gender
0,127000,107000.0,NaN
1,100000,0.0,NaN
2,310000,155000.0,NaN
3,372000,157000.0,NaN
4,157000,0.0,NaN


In [2]:
df = df.drop(['basesalary'], axis=1)
df

,totalyearlycompensation,gender
0,127000,NaN
1,100000,NaN
2,310000,NaN
3,372000,NaN
4,157000,NaN
...,...,...
62637,327000,NaN
62638,237000,NaN
62639,220000,NaN
62640,280000,NaN


In [3]:
df = df.dropna(axis=0, how='any')
df

,totalyearlycompensation,gender
264,140000,Male
265,172000,Male
266,325000,Male
269,150000,Male
270,640000,Male
...,...,...
62610,135000,Male
62611,350000,Male
62612,305000,Female
62613,152000,Male


In [4]:
df.nunique()

totalyearlycompensation    813
gender                       4
dtype: int64

In [5]:
#get rid of software engineer as Title: Senior Software Engineer
df = df[df["gender"].str.contains("Title: Senior Software Engineer") == False]

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(df, dtype=int)
df_dummies

,totalyearlycompensation,gender_Female,gender_Male,gender_Other
264,140000,0,1,0
265,172000,0,1,0
266,325000,0,1,0
269,150000,0,1,0
270,640000,0,1,0
...,...,...,...,...
62610,135000,0,1,0
62611,350000,0,1,0
62612,305000,1,0,0
62613,152000,0,1,0


In [7]:
df_dummies.columns

Index(['totalyearlycompensation', 'gender_Female', 'gender_Male',
       'gender_Other'],
      dtype='object')

In [8]:
# Split our preprocessed data into our features and target arrays
y = df_dummies["totalyearlycompensation"]
X = df_dummies.drop("totalyearlycompensation", axis=1)


In [9]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=2, activation="relu", input_dim=3))

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 2)                 8         
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 11 (44.00 Byte)
Trainable params: 11 (44.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
1011/1011 [==============================] - 2s 1ms/step - loss: -2255273.7500 - accuracy: 0.0000e+00
Epoch 2/10
1011/1011 [==============================] - 1s 1ms/step - loss: -2673419.5000 - accuracy: 0.0000e+00
Epoch 3/10
1011/1011 [==============================] - 1s 1ms/step - loss: -2692807.7500 - accuracy: 0.0000e+00
Epoch 4/10
1011/1011 [==============================] - 1s 1ms/step - loss: -2712992.2500 - accuracy: 0.0000e+00
Epoch 5/10
1011/1011 [==============================] - 1s 1ms/step - loss: -2744058.7500 - accuracy: 0.0000e+00
Epoch 6/10
1011/1011 [==============================] - 1s 1ms/step - loss: -3152078.2500 - accuracy: 0.0000e+00
Epoch 7/10
1011/1011 [==============================] - 1s 1ms/step - loss: -3201912.7500 - accuracy: 0.0000e+00
Epoch 8/10
1011/1011 [==============================] - 1s 1ms/step - loss: -3201911.7500 - accuracy: 0.0000e+00
Epoch 9/10
1011/1011 [==============================] - 1s 1ms/step - loss: -3201911.7500 - accu

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")